In [1]:
!pip install tensorflow tensorflow-gpu opencv-python matplotlib

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached python_version-0.0.2-py2.py3-none-any.whl (3.4 kB)
  Running setup.py clean for tensorflow-gpu
Failed to build tensorflow-gpu


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [18 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\Ayushi_Kansara\AppData\Local\Temp\pip-install-o4b36jrq\tensorflow-gpu_0cb82c9473b54a85a971b9bd12621828\setup.py", line 37, in <module>
          raise Exception(TF_REMOVAL_WARNING)
      Exception:
      
      The "tensorflow-gpu" package has been removed!
      
      Please install "tensorflow" instead.
      
      Other than the name, the two packages have been identical
      since TensorFlow 2.1, or roughly since Sep 2019. For more
      information, see: pypi.org/project/tensorflow-gpu
      
      
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tensorflow-gpu
ERROR: Could

In [5]:
import tensorflow as tf
import os

In [11]:
# gpus = tf.config.experimental.list_physical_devices('CPU')
# len(gpus)

1

In [ ]:
# Avoid OOM errors by setting GPU Memory Consumption Growth 
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [12]:
import cv2
import imghdr

C:\Users\Ayushi_Kansara\AppData\Local\Temp\ipykernel_33132\4232469594.py:2: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


In [13]:
data_dir = "data"
image_exts = ["jpeg", "png", "bmp", "png"]

In [22]:
# import matplotlib.pyplot as plt
# img = cv2.imread(os.path.join('data', 'geto', "Getou.Suguru.full.3251049.jpg"))
# plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [23]:
for img_class in os.listdir(data_dir):    # for gojo and getou 
    for image in os.listdir(os.path.join(data_dir, img_class)):  #for each image
        image_path = os.path.join(data_dir, img_class, image)
        try:
            img = cv2.imread(image_path)   #converts to numpy array pewpew
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except:
            print('Issue with image {}'.format(image_path)) 
            os.remove(image_path)

Image not in ext list data\geto\10704138.jpg.webp
Image not in ext list data\geto\12744138.jpg.webp
Image not in ext list data\geto\7ec6e36d069e2e9da5cac45e1105846811b79f7f.gif
Image not in ext list data\geto\97849_300.webp
Image not in ext list data\geto\geto-getou.gif
Image not in ext list data\geto\getou-getou-suguru.gif
Image not in ext list data\geto\getousuguru-getou.gif
Image not in ext list data\geto\Geto_JJK0.jpg
Image not in ext list data\geto\jujutsu-kaisen-getou.gif
Image not in ext list data\geto\suguru-geto-1.webp
Image not in ext list data\gojo\1000016101_On7jGPEo5y-1024x576.jpeg.webp
Image not in ext list data\gojo\Anime_JujutsuKaisen_Fortnite_1.jpg
Image not in ext list data\gojo\cbb615c2-21a5-4778-87af-d74be31873ca.jpg.webp
Image not in ext list data\gojo\d-to-gojo-satoru-in-jujutsu-kaisen--will-gojo-come-650d1d518c9ea9232205-900.webp
Image not in ext list data\gojo\gojo-satoru.gif
Image not in ext list data\gojo\Gojo_vs_Sukuna.webp
Image not in ext list data\gojo\Juj

In [25]:
# tf.data.Dataset??

Init signature: tf.data.Dataset(variant_tensor)
Source:        
@tf_export("data.Dataset", v1=[])
class DatasetV2(
    collections_abc.Iterable,
    tracking_base.Trackable,
    composite_tensor.CompositeTensor,
    data_types.DatasetV2,
    metaclass=abc.ABCMeta):
  """Represents a potentially large set of elements.

  The `tf.data.Dataset` API supports writing descriptive and efficient input
  pipelines. `Dataset` usage follows a common pattern:

  1. Create a source dataset from your input data.
  2. Apply dataset transformations to preprocess the data.
  3. Iterate over the dataset and process the elements.

  Iteration happens in a streaming fashion, so the full dataset does not need to
  fit into memory.

  Source Datasets:

  The simplest way to create a dataset is to create it from a python `list`:

  >>> dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
  >>> for element in dataset:
  ...   print(element)
  tf.Tensor(1, shape=(), dtype=int32)
  tf.Tensor(2, shape=(), dty

In [26]:
import numpy as np
import matplotlib.pyplot as plt

In [30]:
data = tf.keras.utils.image_dataset_from_directory('data')

Found 162 files belonging to 2 classes.


In [34]:
data_iterator = data.as_numpy_iterator()
batch = data_iterator.next()